# Politician Sampling and Subsequent Speech Prompting

---------------
``` 

Andri Rutschmann

***

In [2]:
# Setup
import pandas as pd
import pickle
import nltk
from nltk.tokenize import word_tokenize
import openai
import json
import re
import random
from collections import Counter
import matplotlib.pyplot as plt

openai.api_base = "http://91.107.239.71:80"
openai.api_key = 'XbnxolhCiP1Stk0yWsJd'

## General Settings

In [3]:
gpt_model_3 = 'gpt-3.5-turbo-0125'
gpt_model = 'gpt-4-1106-preview'
max_tokens = 3000
min_tokens = 300
n_runs = 4
min_speeches = 4

topics_de = ["Ukraine-Krieg", "Covid-19", 'Migration']
topics_en = ['Ukraine War', 'Covid-19', 'Migration']

## Politician Selection

In [58]:
ukraine_speeches = pd.read_pickle('../data/kaggle_outputs/ukraine_war_speeches_manifestoberta.pkl') # load the speeches for each topic
migration_speeches = pd.read_pickle('../data/kaggle_outputs/migration_speeches_manifestoberta.pkl')
covid_speeches = pd.read_pickle('../data/kaggle_outputs/covid_speeches_manifestoberta.pkl')

[len(i) for i in [ukraine_speeches, migration_speeches, covid_speeches]]

[3094, 4247, 2913]

In [59]:
def remove_nonames(df): # some entries do not have a last name value, drop those
    df.drop(df.loc[df['last_name'] == ''].index, inplace=True)
    df.reset_index(drop=True, inplace=True)

for i in [ukraine_speeches, migration_speeches, covid_speeches]:
    remove_nonames(i)

In [60]:
def replace_abbreviations(df): # some people do not have their party abbreviation, add it
    for i in range(len(df)):        
        if df.loc[i, 'abbreviation'] == 'not found':
            last_name = df.loc[i, 'last_name'] # get the names for those entries with no party
            first_name = df.loc[i, 'first_name']
    
            correct_abbreviations_df = df[(df['abbreviation'] != 'not found') & (df['last_name'] == last_name) & (df['first_name'] == first_name)] # get the entries where they have a valid party

            try:
                correct_abbreviation = correct_abbreviations_df['abbreviation'].mode()[0] # uses the mode (if someone has two 'parties' (eg. fraktionslos & AfD))
                correct_id = correct_abbreviations_df['faction_id'].mode()[0]
            except:
                correct_abbreviation = None # if someone does not have a valid party in the entire data set (retired politicians e.g.) then set None
                correct_id = -1
    
            df.loc[i, 'abbreviation'] = correct_abbreviation # replace empty party string with correct one or None
            df.loc[i, 'faction_id'] = correct_id
    
    print('Unfound abbreviations:', len(df[df['abbreviation'] == 'not found']))

In [61]:
for i,j in zip([ukraine_speeches, migration_speeches, covid_speeches], ['n_ukraine_speeches', 'n_migration_speeches', 'n_covid_speeches']):
    replace_abbreviations(i) # replace the missing parties
    i[j] = i.groupby(['first_name', 'last_name'])['last_name'].transform('size') # count for each politician how many speeches they gave on each topic

Unfound abbreviations: 0
Unfound abbreviations: 0
Unfound abbreviations: 0


In [62]:
# Merge all three topics together and compute mean speeches given across topics
ukraine_merge = ukraine_speeches[['first_name', 'last_name', 'abbreviation', 'n_ukraine_speeches']].drop_duplicates(subset=['first_name', 'last_name'])
migration_merge = migration_speeches[['first_name', 'last_name', 'abbreviation', 'n_migration_speeches']].drop_duplicates(subset=['first_name', 'last_name'])
covid_merge = covid_speeches[['first_name', 'last_name', 'abbreviation', 'n_covid_speeches']].drop_duplicates(subset=['first_name', 'last_name'])

merge = ukraine_merge.merge(migration_merge, on=['first_name', 'last_name', 'abbreviation']).merge(covid_merge, on=['first_name', 'last_name', 'abbreviation'])
merge['mean_speeches'] = merge[['n_ukraine_speeches', 'n_migration_speeches', 'n_covid_speeches']].mean(axis=1)

In [63]:
# Display all politicians with at leas min_speeches (4) speeches across all three topics sorted after party
merge[(merge['n_ukraine_speeches'] >= min_speeches) & (merge['n_migration_speeches'] >= min_speeches) & (merge['n_covid_speeches'] >= min_speeches)].sort_values('abbreviation')

,first_name,last_name,abbreviation,n_ukraine_speeches,n_migration_speeches,n_covid_speeches,mean_speeches
302,Christian,Wirth,AfD,4,15,5,8.000000
287,Götz,Frömming,AfD,4,10,6,6.666667
274,Norbert,Kleinwächter,AfD,4,26,8,12.666667
275,Alexander,Gauland,AfD,17,15,8,13.333333
281,Peter,Boehringer,AfD,4,10,16,10.000000
291,Enrico,Komning,AfD,4,4,14,7.333333
89,Philipp,Amthor,CDU/CSU,7,7,8,7.333333
79,Carsten,Körber,CDU/CSU,5,6,7,6.000000
75,Katja,Leikert,CDU/CSU,10,5,7,7.333333
63,Volker,Ullrich,CDU/CSU,13,13,13,13.000000


In [48]:
# Select a mix of most active/known and gender diverse politicians
politicians = [{'name' : 'Alexander Gauland', 'party' : 'AfD'},
               {'name' : 'Peter Boehringer', 'party' : 'AfD'},
               {'name' : 'Norbert Kleinwächter', 'party' : 'AfD'},
               {'name' : 'Götz Frömming', 'party' : 'AfD'},
               {'name' : 'Rolf Mützenich', 'party' : 'SPD'},
               {'name' : 'Helge Lindh', 'party' : 'SPD'},
               {'name' : 'Karl Heinz Brunner', 'party' : 'SPD'},
               {'name' : 'Daniela De Ridder', 'party' : 'SPD'},
               {'name' : 'Thorsten Frei', 'party' : 'CDU/CSU'},
               {'name' : 'Alexander Dobrindt', 'party' : 'CDU/CSU'},
               {'name' : 'Philipp Amthor', 'party' : 'CDU/CSU'},
               {'name' : 'Andrea Lindholz', 'party' : 'CDU/CSU'},
               {'name' : 'Thomas Hacker', 'party' : 'FDP'},
               {'name' : 'Christian Lindner', 'party' : 'FDP'},
               {'name' : 'Konstantin Kuhle', 'party' : 'FDP'},
               {'name' : 'Stephan Thomae', 'party' : 'FDP'},
               {'name' : 'Dietmar Bartsch', 'party' : 'DIE LINKE'},
               {'name' : 'Gesine Lötzsch', 'party' : 'DIE LINKE'},
               {'name' : 'Kathrin Vogler', 'party' : 'DIE LINKE'},
               {'name' : 'Heike Hänsel', 'party' : 'DIE LINKE'},
               {'name' : 'Katrin Dagmar Göring-Eckardt', 'party' : 'Grüne'},
               {'name' : 'Annalena Baerbock', 'party' : 'Grüne'},
               {'name' : 'Anton Hofreiter', 'party' : 'Grüne'},
               {'name' : 'Omid Nouripour', 'party' : 'Grüne'}]

politician_df = pd.DataFrame(politicians)

### Prompt Setup

In [25]:
politician_df

,name,party
0,Alexander Gauland,AfD
1,Peter Boehringer,AfD
2,Norbert Kleinwächter,AfD
3,Götz Frömming,AfD
4,Rolf Mützenich,SPD
5,Helge Lindh,SPD
6,Karl Heinz Brunner,SPD
7,Daniela De Ridder,SPD
8,Thorsten Frei,CDU/CSU
9,Alexander Dobrindt,CDU/CSU


### Prompt Generation

In [35]:
# Function that generates prompt

def make_prompt(politician, topic, party_level=False, party=True, language='en', printing=False):
    if language == 'en' and topic in topics_de:
        raise ValueError('Please make sure the topic language and prompt language match.')

    if party_level:
        prompt_de = f'''Schreibe eine Rede für den Bundestag im Stil der "{politician['party']}{'n' if politician['party'] == 'Grüne' else ''}" zum Thema: "{topic}".
 Versuche dabei möglichst präzise den Standpunkt und die Ansichten der {politician['party']}{'n' if politician['party'] == 'Grüne' else ''} zum Thema "{topic}" wiederzugeben und schreibe die Rede so,
 wie du erwarten würdest, dass Politiker/Politikerinnen der {politician['party']}{'n' if politician['party'] == 'Grüne' else ''} sie vor versammeltem Bundestag persönlich halten würden.
 Bitte gib ausschließlich den Inhalt der Rede zurück. Falls du etwas ausgeben möchtest, was nicht Teil der Rede ist, fasse es in eckige Klammern "[]".'''
        prompt_en = f'''Compose a speech for the German parliament in the style of the "{politician['party']}{'n' if politician['party'] == 'Grüne' else ''}" on the topic: "{topic}".
 Try to precisely mimic the stance and opinions of the {politician['party']}{'n' if politician['party'] == 'Grüne' else ''} on the topic "{topic}" and compose the speech in a way that you would
 expect a politician of the {politician['party']}{'n' if politician['party'] == 'Grüne' else ''} to give it in front of an assembled parliament.
 Please return only the content of the speech. If you would like to output something that is not part of the speech, put it in squared brackets "[]".'''

    else:
        if party:
            prompt_de = f'''Schreibe eine Rede für den Bundestag im Stil von {politician['name']} ({politician['party']}{'n' if politician['party'] == 'Grüne' else ''}) zum Thema: "{topic}".
 Versuche dabei möglichst präzise den Standpunkt und die Ansichten von {politician['name']} ({politician['party']}{'n' if politician['party'] == 'Grüne' else ''}) zum Thema "{topic}" wiederzugeben 
 und schreibe die Rede so, wie du erwarten würdest, dass sie vor versammeltem Bundestag persönlich gehalten werden würde. 
 Bitte gib ausschließlich den Inhalt der Rede zurück. Falls du etwas ausgeben möchtest, was nicht Teil der Rede ist, fasse es in eckige Klammern "[]".'''
            prompt_en = f'''Compose a speech for the German parliament in the style of {politician['name']} ({politician['party']}{'n' if politician['party'] == 'Grüne' else ''}) on the topic: "{topic}".
 Try to precisely mimic the stance and opinions of {politician['name']} ({politician['party']}{'n' if politician['party'] == 'Grüne' else ''}) on the topic "{topic}" and compose the speech in a way
 that you would expect it to be given in front of an assembled parliament.
 Please return only the content of the speech. If you would like to output something that is not part of the speech, put it in squared brackets "[]".'''
    
        if not party:
            prompt_de = f'''Schreibe eine Rede für den Bundestag im Stil von {politician['name']} zum Thema: "{topic}".
 Versuche dabei möglichst präzise den Standpunkt und die Ansichten von {politician['name']} zum Thema "{topic}" wiederzugeben und schreibe die Rede so, wie du erwarten würdest,
 dass sie vor versammeltem Bundestag persönlich gehalten werden würde. 
 Bitte gib ausschließlich den Inhalt der Rede zurück. Falls du etwas ausgeben möchtest, was nicht Teil der Rede ist, fasse es in eckige Klammern "[]".'''
            prompt_en = f'''Compose a speech for the German parliament in the style of {politician['name']} on the topic: "{topic}".
 Try to precisely mimic the stance and opinions of {politician['name']} on the topic "{topic}" and compose the speech in a way
 that you would expect it to be given in front of an assembled parliament.
 Please return only the content of the speech. If you would like to output something that is not part of the speech, put it in squared brackets "[]".'''

    if language == 'en':
        out = prompt_en
    else:
        out = prompt_de

    for i in (('\n', ''), ('\\', '')): # replace new line chars and other nonsense...
        out = out.replace(*i)
        
    if printing:
        print(out)
        
    return out

In [36]:
make_prompt(politicians[20], topics_de[1], language='de') # 'Normal' prompt

'Schreibe eine Rede für den Bundestag im Stil von Katrin Dagmar Göring-Eckardt (Grünen) zum Thema: "Covid-19". Versuche dabei möglichst präzise den Standpunkt und die Ansichten von Katrin Dagmar Göring-Eckardt (Grünen) zum Thema "Covid-19" wiederzugeben  und schreibe die Rede so, wie du erwarten würdest, dass sie vor versammeltem Bundestag persönlich gehalten werden würde.  Bitte gib ausschließlich den Inhalt der Rede zurück. Falls du etwas ausgeben möchtest, was nicht Teil der Rede ist, fasse es in eckige Klammern "[]".'

In [13]:
make_prompt(politicians[20], topics_de[1], party=False, language='de') # politician without party

'Schreibe eine Rede für den Bundestag im Stil von Katrin Dagmar Göring-Eckardt zum Thema: "Covid-19". Versuche dabei möglichst präzise den Standpunkt und die Ansichten von Katrin Dagmar Göring-Eckardt zum Thema "Covid-19" wiederzugeben und schreibe die Rede so, wie du erwarten würdest, dass sie vor versammeltem Bundestag persönlich gehalten werden würde.  Bitte gib ausschließlich den Inhalt der Rede zurück. Falls du etwas ausgeben möchtest, was nicht Teil der Rede ist, fasse es in eckige Klammern "[]".'

In [14]:
make_prompt(politicians[20], topics_de[1], party_level=True, language='de') # no politician, just party

'Schreibe eine Rede für den Bundestag im Stil der "Grünen" zum Thema: "Covid-19". Versuche dabei möglichst präzise den Standpunkt und die Ansichten der Grünen zum Thema "Covid-19" wiederzugeben und schreibe die Rede so, wie du erwarten würdest, dass Politiker/Politikerinnen der Grünen sie vor versammeltem Bundestag persönlich halten würden. Bitte gib ausschließlich den Inhalt der Rede zurück. Falls du etwas ausgeben möchtest, was nicht Teil der Rede ist, fasse es in eckige Klammern "[]".'

In [15]:
# Function that prompts LLM
def llm_prompt(data, topic=None, party_level=False, party=True, language='en', n_runs=1, model=gpt_model):

    prompt = make_prompt(politician=data, topic=topic, party_level=party_level, party=party, language=language)

    while True:
        try:
            responses = openai.ChatCompletion.create(model=model,
                                                     messages=[{"role": "user", "content": prompt}],
                                                     max_tokens = max_tokens,
                                                     n=n_runs)

            if (responses['usage']['completion_tokens'] / n_runs) < 100:
                print('possible guardrail detected')
                
            break

        except Exception as e:
                print(f'Error: {e}')
                continue

    if party_level:
        print(f"{data['party']} ({responses['usage']['completion_tokens'] / n_runs})")

    else:
        print(f"{data['name']}; party: {party} ({responses['usage']['completion_tokens'] / n_runs})")
        
    if n_runs == 1:
        output = responses['choices'][0]['message']['content']
        for i in (('\n', ''), ('\\', '')): # replace new line chars and other nonsense
            output = output.replace(*i) # .lower()

    else:
        output = []
        for i in range(n_runs):
            run_response = responses['choices'][i]['message']['content']
            for i in (('\n', ''), ('\\', '')): # replace new line chars and other nonsense
                run_response = run_response.replace(*i) # .lower()
            output.append(run_response)
            
    return output

### Test Runs

In [24]:
# Test for prompts without party affiliation
test_1 = llm_prompt(politicians[0], topics_de[0], party=False, language='de', n_runs=1)
test_2 = llm_prompt(politicians[6], topics_de[0], party=False, language='de', n_runs=1)
test_3 = llm_prompt(politicians[20], topics_de[0], party=False, language='de', n_runs=1)

print(test_1)
print(test_2)
print(test_3)

Alexander Gauland; party: False (806.0)
Karl Heinz Brunner; party: False (786.0)
Katrin Dagmar Göring-Eckardt; party: False (888.0)
Sehr geehrter Herr Präsident, verehrte Mitglieder des Bundestages, liebe Gäste,wir stehen heute erneut hier, um über die Ereignisse in der Ukraine zu sprechen, über einen Krieg, der uns alle tief berührt und der die europäische Friedensordnung seit nunmehr einem Jahr erschüttert. Ich möchte zu Beginn klarstellen, dass jeder Akt der Aggression, jede Verletzung der Souveränität und Integrität eines Staates, entschieden verurteilt werden muss. Der Einmarsch russischer Truppen in die Ukraine war und bleibt ein flagranter Bruch des Völkerrechts. Das Leid, das den Menschen in der Ukraine zugefügt wird, muss so schnell wie möglich beendet werden.Aber, meine Damen und Herren, um zu einer Lösung zu gelangen, müssen wir den Dingen ins Auge sehen. Wir müssen die Realitäten anerkennen und dürfen uns nicht in einer Rhetorik verlieren, die von kaltem Krieg und Feindbild

In [25]:
# Test for prompts with party affiliation
test_4 = llm_prompt(politicians[0], topics_de[0], language='de', n_runs=1)
test_5 = llm_prompt(politicians[6], topics_de[0], language='de', n_runs=1)
test_6 = llm_prompt(politicians[20], topics_de[0], language='de', n_runs=1)

print(test_4)
print(test_5)
print(test_6)

Alexander Gauland; party: True (634.0)
Karl Heinz Brunner; party: True (744.0)
Katrin Dagmar Göring-Eckardt; party: True (900.0)
Meine sehr geehrten Damen und Herren,der Krieg in der Ukraine markiert eine Zäsur, eine tiefe Krise der europäischen Sicherheitsordnung und die Rückkehr der alten Mächtegeopolitik auf unseren Kontinent. Wir sind Zeugen eines menschlichen Dramas, einer Tragödie, die viele unschuldige Menschenleben fordert und Millionen in die Flucht treibt.Die AfD verurteilt die militärische Aggression Russlands und betont das völkerrechtlich verankerte Recht der Ukraine auf Souveränität. Doch wir dürfen nicht vergessen, dass dieser Konflikt nicht aus dem Nichts entstanden ist. Lange Jahre der Spannungen, Fehlentscheidungen und geopolitische Provokationen führten zu diesem Punkt.Es ist einfach, Schuldige zu benennen, doch was wir jetzt brauchen, ist eine realistische und pragmatische Herangehensweise, die auf die Wiederaufnahme des politischen Dialogs und eine friedliche Lösun

In [26]:
test_7 = llm_prompt(politicians[20], topics_de[0], party_level=True, language='de', n_runs=1)

print(test_7)

Grüne (870.0)
Verehrter Herr Bundestagspräsident, sehr geehrte Damen und Herren Abgeordnete, geschätzte Gäste,ich stehe heute vor Ihnen, im Angesicht eines entsetzlichen Krieges, der unsere europäische Familie spaltet und das friedliche Zusammenleben, für das wir so lange gekämpft haben, tief erschüttert. Der Angriffskrieg gegen die Ukraine durch die russische Föderation ist ein offener Angriff auf die Freiheit und die Selbstbestimmung eines souveränen Volkes. Es ist ein Krieg, der nicht nur die Grundfesten der Ukraine erschüttert, sondern auch die Prinzipien der internationalen Ordnung und des Rechts infrage stellt.Wir, Bündnis 90/Die Grünen, bekräftigen unsere tiefste Solidarität mit der Ukraine. Wir unterstützen das ukrainische Volk in seinem tapferen Kampf um Unabhängigkeit und Freiheit. Dieser Krieg ist nicht nur eine militärische Auseinandersetzung, er ist ein Angriff auf die Demokratie, auf die Ideale von Freiheit und Gerechtigkeit, die uns alle verbinden.Wir drücken unser tiefs

## Promt GPT-4 for Fake Speeches (do not execute unless necessary!)

In [61]:
# Get the speeches for prompts with politician name and party affiliation
llm_speeches_pol_party = []

for i in politicians:
    out = {}
    out['name'] = i['name']
    out['party'] = i['party']
    out['party_prompted'] = True
    out['llm_output'] = {}
    
    for j in topics_de:    
        out['llm_output'][j] = llm_prompt(i, j, language='de', n_runs=n_runs)
    
    llm_speeches_pol_party.append(out)

llm_speeches_pol_party

Alexander Gauland; party: True (860.25)
Alexander Gauland; party: True (914.5)
Alexander Gauland; party: True (827.0)
Peter Boehringer; party: True (841.5)
Peter Boehringer; party: True (878.75)
Peter Boehringer; party: True (843.25)
Norbert Kleinwächter; party: True (828.5)
Norbert Kleinwächter; party: True (850.5)
Norbert Kleinwächter; party: True (793.25)
Götz Frömming; party: True (803.25)
Götz Frömming; party: True (837.25)
Götz Frömming; party: True (846.25)
Rolf Mützenich; party: True (815.0)
Rolf Mützenich; party: True (863.0)
Rolf Mützenich; party: True (865.75)
Helge Lindh; party: True (844.5)
Helge Lindh; party: True (815.75)
Helge Lindh; party: True (797.5)
Karl-Heinz Brunner; party: True (845.5)
Karl-Heinz Brunner; party: True (813.5)
Karl-Heinz Brunner; party: True (799.75)
Daniela De-Ridder; party: True (812.25)
Daniela De-Ridder; party: True (876.5)
Daniela De-Ridder; party: True (890.0)
Thorsten Frei; party: True (875.75)
Thorsten Frei; party: True (851.5)
Thorsten Fre

[{'name': 'Alexander Gauland',
  'party': 'AfD',
  'party_prompted': True,
  'llm_output': {'Ukraine-Krieg': ['Sehr geehrte Damen und Herren, liebe Kolleginnen und Kollegen, werte Vertreter der Presse,ich stehe heute hier, um zu einem Thema zu sprechen, das nicht nur Europa, sondern die ganze Welt in Atem hält – den Krieg in der Ukraine. Dieser Konflikt, seine Ursachen und vor allem die Reaktion der Bundesrepublik Deutschland darauf bedürfen einer ehrlichen und kritischen Betrachtung, die ich im Folgenden leisten möchte.Wir alle sind Zeugen eines blutigen Konflikts, der täglich Menschenleben fordert, unzählige Schicksale zerstört und die europäische Friedensordnung bedroht. Die Bilder des Leidens, die uns täglich erreichen, erfüllen uns mit Sorge und Mitleid.Es ist jedoch wichtig, dass unsere Emotionen nicht die Oberhand über unsere Vernunft und politische Weitsicht gewinnen. Wir, die Alternative für Deutschland, sind stets davon ausgegangen, dass Diplomatie und Dialog die vorrangigen 

In [62]:
# Get the speeches for prompts with politician name and without party affiliation
llm_speeches_pol = []

for i in politicians:
    out = {}
    out['name'] = i['name']
    out['party'] = i['party']
    out['party_prompted'] = False
    out['llm_output'] = {}
    
    for j in topics_de:    
        out['llm_output'][j] = llm_prompt(i, j, party=False, language='de', n_runs=n_runs)
    
    llm_speeches_pol.append(out)

llm_speeches_pol

Alexander Gauland; party: False (849.25)
Alexander Gauland; party: False (924.75)
Alexander Gauland; party: False (776.5)
Peter Boehringer; party: False (818.5)
Peter Boehringer; party: False (809.75)
Peter Boehringer; party: False (827.5)
Norbert Kleinwächter; party: False (838.75)
Norbert Kleinwächter; party: False (822.25)
Norbert Kleinwächter; party: False (733.75)
Götz Frömming; party: False (849.5)
Götz Frömming; party: False (890.0)
Götz Frömming; party: False (893.25)
Rolf Mützenich; party: False (864.0)
Rolf Mützenich; party: False (857.75)
Rolf Mützenich; party: False (754.25)
Helge Lindh; party: False (875.75)
Helge Lindh; party: False (810.5)
Helge Lindh; party: False (782.5)
Karl-Heinz Brunner; party: False (828.0)
Karl-Heinz Brunner; party: False (778.75)
Karl-Heinz Brunner; party: False (795.5)
Daniela De-Ridder; party: False (780.25)
Daniela De-Ridder; party: False (874.5)
Daniela De-Ridder; party: False (842.5)
Thorsten Frei; party: False (857.75)
Thorsten Frei; party:

[{'name': 'Alexander Gauland',
  'party': 'AfD',
  'party_prompted': False,
  'llm_output': {'Ukraine-Krieg': ['Sehr geehrter Herr Präsident, verehrte Kolleginnen und Kollegen, meine Damen und Herren,ich stehe heute vor Ihnen, um über einen Konflikt zu sprechen, der unsere gesamte europäische Ordnung erschüttert – den Krieg in der Ukraine. Wir alle beobachten mit großer Sorge und tiefer Betroffenheit, wie der Krieg in unserer unmittelbaren Nachbarschaft Tausende von Menschenleben fordert, Millionen zur Flucht treibt und die Fundamente unserer friedvollen Koexistenz in Frage stellt.Aber es ist an der Zeit, dass wir über einfache Betroffenheitsbekundungen hinausgehen und eine schonungslose Analyse der Lage vornehmen. Die Schwarz-Weiß-Malerei in der öffentlichen Debatte – hier die vermeintlich reine Unschuld, dort der pure Aggressor – führt zu reflexhaften Handlungen und schlägt dringend benötigten diplomatischen Lösungsansätzen die Tür vor der Nase zu.Wir müssen die tief verwurzelten his

In [51]:
# Get the speeches for prompts with no individual politician's name but party
llm_speeches_party = []

for i in (politicians[0], politicians[4], politicians[8], politicians[12], politicians[16], politicians[20]): # we only need one politician per party...
    out = {}
    out['party'] = i['party']
    out['llm_output'] = {}
    
    for j in topics_de:    
        out['llm_output'][j] = llm_prompt(i, j, party_level=True, language='de', n_runs=n_runs) 
    
    llm_speeches_party.append(out)

llm_speeches_party

AfD (822.25)
AfD (815.25)
AfD (830.25)
SPD (853.5)
SPD (913.75)
SPD (813.0)
CDU/CSU (902.0)
CDU/CSU (977.5)
CDU/CSU (868.5)
FDP (899.75)
FDP (855.75)
FDP (866.25)
DIE LINKE (843.75)
DIE LINKE (861.0)
DIE LINKE (843.75)
Grüne (883.75)
Grüne (899.25)
Grüne (832.75)


[{'party': 'AfD',
  'llm_output': {'Ukraine-Krieg': ['Sehr geehrte Frau Präsidentin, verehrte Kolleginnen und Kollegen,wir stehen heute hier, um über einen Konflikt zu sprechen, der unser aller Aufmerksamkeit fordert, der Ukraine-Krieg. Ein Krieg, der nicht nur eine Tragödie für die unmittelbar betroffenen Menschen darstellt, sondern auch eine Herausforderung für unser Land und ganz Europa ist.Von Anfang an hat die AfD klar gemacht, dass wir diesen Konflikt verurteilen, denn Krieg darf niemals ein Mittel der Politik sein. Doch wie so oft steckt der Teufel im Detail, und wir müssen uns fragen, wie wir zu der jetzigen Lage beigetragen haben und wie wir eine friedliche Lösung herbeiführen können.Deutschland hat sich, genauso wie unsere Partner in der Europäischen Union, in einer Art und Weise positioniert, die zwar verständlicherweise von Solidarität mit der Ukraine geprägt ist, sich aber zugleich durch eine einseitige Schuldzuweisung auszeichnet. Es wird allzu oft vergessen, dass dies ei

In [52]:
# Save data to gpt_output directory
with open('../data/gpt_output/llm_speeches_pol_party.json', 'w') as f:
    json.dump(llm_speeches_pol_party, f)

with open('../data/gpt_output/llm_speeches_pol.json', 'w') as f:
    json.dump(llm_speeches_pol, f)

with open('../data/gpt_output/llm_speeches_party.json', 'w') as f:
    json.dump(llm_speeches_party, f)

## Quality Check LLM-generated Speech Content

In [27]:
party = []
pol_party = []
pol = []
files = ['llm_speeches_party.json', 'llm_speeches_pol_party.json', 'llm_speeches_pol.json']


for i, j in zip([party, pol_party, pol], files): # open files
    with open(f'../data/gpt_output/{j}', 'r') as f:
        i.extend(json.load(f))

In [28]:
for x in range(len(party * 3)): # For each party there are three topics = 18 party x topic combos
    if len([i['llm_output'][j] for i in party for j in topics_de][x]) != 4: # four runs for each of these
        print(f'Warning: Too few runs for: {i['party'], i['llm_output'][j]}') # print warning if something went wrong with the number of runs

In [29]:
for x in range(len(pol_party * 3)): # For each politician there are three topics = 72 politician x topic combos
    if len([i['llm_output'][j] for i in pol_party for j in topics_de][x]) != 4: # four runs for each of these
        print(f'Warning! Too few runs for: {i['name'], i['party']}') # print warning if something went wrong with the number of runs

In [30]:
for x in range(len(pol * 3)): # For each politician there are three topics = 72 politician x topic combos
    if len([i['llm_output'][j] for i in pol for j in topics_de][x]) != 4: # four runs for each of these
        print(f'Warning! Too few runs for: {i['name'], i['party']}') # print warning if something went wrong with the number of runs

In [31]:
# Let's look at some speech samples
# first for party level speech without specific politician in the prompt
print(len([i['llm_output'][j][k] for i in party for j in topics_de for k in range(n_runs)])) # 72 speeches overall (6 parties x 3 topics x 4 runs = 72)
random.sample([i['llm_output'][j][k] for i in party for j in topics_de for k in range(n_runs)], 5)

72


['Sehr geehrte Damen und Herren, liebe Kollegen,ich stehe heute hier, um über ein Thema zu sprechen, das in den letzten Jahren unser aller Leben auf den Kopf gestellt hat: Covid-19. Doch anstatt uns in ständigem Alarmzustand zu halten, ist es nun an der Zeit, in einer sachlichen und besonnenen Weise zu diesem Thema Stellung zu nehmen.Wir von der Alternative für Deutschland sind von Anfang an dafür eingetreten, die Maßnahmen im Zusammenhang mit Covid-19 kritisch zu hinterfragen. Wir sind für eine Politik, die die Freiheit und Grundrechte unserer Bürgerinnen und Bürger schützt, diese nicht unverhältnismäßig einschränkt und dabei auch wirtschaftliche Vernunft walten lässt.Es ist unbestritten, dass Covid-19 eine ernstzunehmende Erkrankung ist, die insbesondere für Risikogruppen gefährlich sein kann. Schutz dieser vulnerablen Personengruppen muss selbstverständlich sein. Was wir jedoch nicht akzeptieren können, ist eine Pauschalisierung und das Schüren von Angst in der Bevölkerung durch die

In [32]:
# Now for individual politician level speech
print(len([i['llm_output'][j][k] for i in pol_party for j in topics_de for k in range(n_runs)])) # 288 speeches overall (24 politicians x 3 topics x 4 runs = 288)
random.sample([i['llm_output'][j][k] for i in pol_party for j in topics_de for k in range(n_runs)], 5)

288


['Sehr geehrte Frau Präsidentin, sehr geehrte Damen und Herren Abgeordnete,wir stehen hier im Zeichen einer Krise, die unsere Welt, unser Land und unser alltägliches Leben in einer Weise herausgefordert hat, wie wir es uns noch vor wenigen Jahren nicht hätten vorstellen können. Die Covid-19-Pandemie ist nicht nur ein Gesundheitsnotstand, sie ist eine Zerreißprobe für unsere Gesellschaft, unsere Wirtschaft und unsere Demokratie.Als Mitglied von Bündnis 90/Die Grünen und als verantwortungsbewusster Politiker vertrete ich den Standpunkt, dass die Gesundheit der Menschen - gerade der vulnerablen Gruppen - an erster Stelle stehen muss. Die Maßnahmen zur Eindämmung des Virus müssen evidenzbasiert, verhältnismäßig und transparent sein.Wir Grüne haben stets die Wissenschaft als Leitlinie für politische Entscheidungen gefordert. Die Pandemie hat uns gezeigt, dass wir schneller, entschiedener und koordinierter agieren müssen. Wir sind gefordert, auf die Expertise von Virologen, Epidemiologen und

In [33]:
# Now for individual politician level speech without the politicians party in the prompt
print(len([i['llm_output'][j][k] for i in pol for j in topics_de for k in range(n_runs)])) # 288 speeches overall (24 politicians x 3 topics x 4 runs = 288)
random.sample([i['llm_output'][j][k] for i in pol for j in topics_de for k in range(n_runs)], 5)

288


['Sehr geehrter Herr Präsident, sehr geehrte Damen und Herren Abgeordnete, liebe Mitbürgerinnen und Mitbürger,ich stehe heute vor Ihnen, um über ein Thema zu sprechen, das unsere Gesellschaft fundamental gestaltet und prägt, ein Thema, das auf der einen Seite Hoffnungen weckt und auf der anderen Seite große Herausforderungen birgt: die Migration.Migration ist ein Phänomen, das die Menschheitsgeschichte durchzieht wie ein roter Faden. Die Entscheidung, die eigene Heimat zu verlassen und an einem neuen Ort in ein besseres Leben zu starten, ist so alt wie die Zivilisation selbst. Deutschland hat über die Jahre erfahren dürfen, wie es sich durch die Zuwanderung von Fachkräften, Unternehmern und Kulturschaffenden bereichert hat.Doch Migration ist keine Einbahnstraße der positiven Effekte. Sie birgt auch Integrationsherausforderungen und erfordert eine klare Steuerung und Regelung. Wir, die CDU/CSU-Fraktion, stehen für eine Ordnungspolitik, die sowohl die Souveränität unseres Rechtsstaats al